# Education ...

Project descriptions

## Step 1: Loading and exploring the dataset

In [19]:
import pandas as pd
# read in the mathematics dataset
maths_df = pd.read_csv("student-mat.csv", sep=";")
print(maths_df.head())

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other ...   
3     GP   F   15       U     GT3       T     4     2   health  services ...   
4     GP   F   16       U     GT3       T     3     3    other     other ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1      5        3      3     1     1      3        4   5   5   6  
2      4        3      2     2     3      3       10   7   8  10  
3      3        2      2     1     1      5        2  15  14  15  
4      4        3      2     1     2      5        4   6  10  10  

[5 rows x 33 columns]


## Step 2: Pre-processing

In [13]:
# Check for missing values
print(maths_df.isnull().values.any())

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other ...   
3     GP   F   15       U     GT3       T     4     2   health  services ...   
4     GP   F   16       U     GT3       T     3     3    other     other ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1      5        3      3     1     1      3        4   5   5   6  
2      4        3      2     2     3      3       10   7   8  10  
3      3        2      2     1     1      5        2  15  14  15  
4      4        3      2     1     2      5        4   6  10  10  

[5 rows x 33 columns]
False


G3 (final year grade) was calculated using G1 and G2 (1st and 2nd period grades). We will use G3 as the responce variable and later on as the class variable by converitng it to a discrete scale. Since G3 is correlated with G1 and G2, the period grade variables will not be used as predictors in this analysis.

## Step 3: Regression

In [54]:
#maths_df_cat = pd.get_dummies(maths_df)
maths_df = maths_df.drop(['G1', 'G2'], axis = 1)



In [55]:
maths_df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,yes,no,5,4,2,1,2,5,10,15
6,GP,M,16,U,LE3,T,2,2,other,other,...,yes,no,4,4,4,1,1,3,0,11
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,no,no,4,1,4,1,1,1,6,6
8,GP,M,15,U,LE3,A,3,2,services,other,...,yes,no,4,2,2,1,1,1,0,19
9,GP,M,15,U,GT3,T,3,4,other,other,...,yes,no,5,5,1,1,1,5,0,15


In [37]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(maths_df_cat.drop(['G3', 'G1', 'G2'], axis=1), maths_df_cat['G3'], test_size=0.2, random_state=42)
mod = linear_model.LinearRegression()
model = mod.fit(train_x, train_y)




In [42]:
pred = model.predict(test_x)

In [49]:
df = pd.DataFrame({'Actual': test_y, 'Predicted': pred})
from sklearn import metrics
import numpy as np

#df.head(25)

print('Mean Absolute Error:', metrics.mean_absolute_error(test_y, pred))  
print('Mean Squared Error:', metrics.mean_squared_error(test_y, pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y, pred)))

print(np.mean(maths_df_cat["G3"]))

Mean Absolute Error: 3.3956716030458862
Mean Squared Error: 17.606221670591378
Root Mean Squared Error: 4.195976843428879
10.415189873417722


## Clustering